**Set environment**

In [8]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [9]:
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

echo +++++++++++++++++++++++++++++++++++
ls -lh ${FD_REG}/${FN_REG}
cat ${FD_REG}/${FN_REG} | wc -l
echo

echo +++++++++++++++++++++++++++++++++++
head ${FD_REG}/${FN_REG}
echo

echo +++++++++++++++++++++++++++++++++++
tail ${FD_REG}/${FN_REG}
echo

+++++++++++++++++++++++++++++++++++
-rw-rw-r-- 1 kk319 reddylab 5.7M Mar  6 16:02 /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
247520

+++++++++++++++++++++++++++++++++++
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613
chr1	30803	31072
chr1	101603	101849
chr1	115411	115986
chr1	118518	118743
chr1	136071	137429

+++++++++++++++++++++++++++++++++++
chrX	155966947	155967395
chrX	155971561	155972273
chrX	155986274	155987619
chrX	155997267	155998045
chrX	156000382	156003205
chrX	156009687	156010227
chrX	156016391	156016836
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422



In [10]:
ASSAY=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/*WGS*.bed.gz | xargs -n 1 basename

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count
A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz


In [11]:
FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
echo Total: ${#FP_INPS[@]}

Total: 7


**RUN**

In [12]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.%a.txt \
    --array 0-6 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output directory
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input

FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_REG=${FD_REG}/${FN_REG}

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
zcat  ${FP_INP} | head -n 5 
echo

echo "Input  file: " ${FP_REG}
echo
echo "Show the first few lines of the input file"
cat  ${FP_REG} | head -n 3 
echo
echo "Show the last few lines of the input file"
cat  ${FP_REG} | tail -n 3 
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### Show I/O file
echo
echo "Output file: " ${FP_OUT}
echo
echo "Show the first few lines of the output file"
zcat ${FP_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29577923


**Check results**

In [13]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.0.txt

Hostname:           x2-08-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-19-23+13:31:50

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/covera

In [14]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.6.txt

Hostname:           x3-02-1.genome.duke.edu
Slurm Array Index:  6
Time Stamp:         03-19-23+13:31:50

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10537	11067	chr1_10537_11067	1	.
chr1	14408	14799	chr1_14408_14799	1	.
chr1	14498	14937	chr1_14498_14937	1	.
chr1	14676	15184	chr1_14676_15184	1	.
chr1	14684	15162	chr1_14684_15162	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/cover